In [ ]:
# # 改spark buffer 設定
# spark.conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
# spark.conf.set("spark.kryoserializer.buffer.max", "128m")

In [1]:
spark.conf.get("spark.kryoserializer.buffer.max")

'512m'

In [2]:
import os
import numpy as np
import pandas as pd

from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.window import *
import pyspark.pandas as ps

In [3]:
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", True)
ps.set_option("compute.default_index_type", "distributed")

In [4]:
spark

# 一個月的資料

In [5]:
customers = spark.read.option('header','true').parquet('/user/HM_parquet/customers.parquet')
trainOneMonth = spark.read.option('header','true').parquet('/user/HM_parquet/train_one_month.parquet')
valOneMonth = spark.read.option('header','true').parquet('/user/HM_parquet/val_one_month.parquet')

In [ ]:
customers.columns

In [ ]:
transaction.columns

In [11]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.feature import StringIndexer, IndexToString

In [14]:
# 將customer_id的字串轉為數字(buffer要增加到512m)
toIndex = StringIndexer(inputCol="customer_id", outputCol="customer_index").fit(customers)
indexed = toIndex.transform(customers)
indexed.show()

2022-03-13 14:28:57,106 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 111.2 MiB
2022-03-13 14:29:01,715 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 111.2 MiB
2022-03-13 14:29:09,213 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 111.2 MiB


+--------------------+----+------+------------------+----------------------+----+--------------------+--------------+
|         customer_id|  FN|Active|club_member_status|fashion_news_frequency| age|         postal_code|customer_index|
+--------------------+----+------+------------------+----------------------+----+--------------------+--------------+
|00000dbacae5abe5e...|null|  null|            ACTIVE|                  NONE|  49|52043ee2162cf5aa7...|           0.0|
|0000423b00ade9141...|null|  null|            ACTIVE|                  NONE|  25|2973abc54daa8a5f8...|           1.0|
|000058a12d5b43e67...|null|  null|            ACTIVE|                  NONE|  24|64f17e6a330a85798...|           2.0|
|00005ca1c9ed5f514...|null|  null|            ACTIVE|                  NONE|  54|5d36574f52495e81f...|           3.0|
|00006413d8573cd20...| 1.0|   1.0|            ACTIVE|             Regularly|  52|25fa5ddee9aac01b3...|           4.0|
|000064249685c1155...|null|  null|                  |   

In [10]:
# 將customer_index轉回原本的id
toId = IndexToString(inputCol="customer_index", outputCol="customer_index_id")
itd = inverter.transform(indexed)
itd.show()

2022-03-13 14:26:04,731 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 111.2 MiB
2022-03-13 14:26:08,668 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 111.2 MiB
2022-03-13 14:26:16,741 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 111.2 MiB


+--------------------+----+------+------------------+----------------------+----+--------------------+--------------+--------------------+
|         customer_id|  FN|Active|club_member_status|fashion_news_frequency| age|         postal_code|customer_index|   customer_index_id|
+--------------------+----+------+------------------+----------------------+----+--------------------+--------------+--------------------+
|00000dbacae5abe5e...|null|  null|            ACTIVE|                  NONE|  49|52043ee2162cf5aa7...|           0.0|00000dbacae5abe5e...|
|0000423b00ade9141...|null|  null|            ACTIVE|                  NONE|  25|2973abc54daa8a5f8...|           1.0|0000423b00ade9141...|
|000058a12d5b43e67...|null|  null|            ACTIVE|                  NONE|  24|64f17e6a330a85798...|           2.0|000058a12d5b43e67...|
|00005ca1c9ed5f514...|null|  null|            ACTIVE|                  NONE|  54|5d36574f52495e81f...|           3.0|00005ca1c9ed5f514...|
|00006413d8573cd20...| 1.0|